In [1]:
import sys
from typing import Literal

import numpy as np
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go
import tensorflow as tf

from tensorflow.keras.layers import GRU, Dense, Embedding, SimpleRNN, StringLookup
from tensorflow.keras.models import Model

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from equation_discover import *

2023-12-26 20:50:47.628068: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-26 20:50:47.630675: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-26 20:50:47.662663: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-26 20:50:47.662689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-26 20:50:47.663847: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
sampler = RNNSampler(BASE_TOKENS, 16, 1)
loss_func = SymbolicLoss()

In [3]:
X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())


X = pandas_to_tensor(X)
y = tf.convert_to_tensor(y, dtype=tf.float32)
constants = tf.Variable([0.0, 0.0, 0.0])

# Test sampler

In [ ]:
with tf.GradientTape() as tape:
    sequences, lengths, entropies, log_probs = sampler.sample(1)

tape.gradient(log_probs, sampler.variables)

In [ ]:
with tf.GradientTape() as tape:
    sequences, lengths, entropies, log_probs = sampler.sample(10)
    y_pred = []
    for sequence, length in zip(sequences, lengths):
        tree = Node.from_sequence(sequence[:length], BASE_TOKENS)
        y_pred.append(tree.tf_eval(X, constants=constants))
    y_pred = tf.stack(y_pred)
    loss = loss_func(y, y_pred, entropies, log_probs)
    print([var.name for var in tape.watched_variables()])


grads = tape.gradient(loss, sampler.variables)

# Test sampler